# Introduction

This notebook aime is to test different technique and used cased focused on video understanding.

First, being able to get structure description as it is something that could highly be valuable for the HR people. In order to get enough context with comparable elements of analyses.
The second one focus on a more quantitative approach that would help to directly compare quantidate. We could even think about a specific formula to "rate" the interaction.
The last call focus on Identify congruence and improvement. I believe that every attend should be able learning and developping human capabilities. So it could be a base to give feadback to the user.


Gen AI Capabilities used
- few shot prompting
- Video understanding
- Structure output

# Prepare the environment

In [1]:
!pip install -Uq "google-genai==1.7.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 4.4 MB/s eta 0:00:00


In [2]:
import os
import os.path
import base64

from google import genai
from google.genai import types

from IPython.display import Markdown, display
from kaggle_secrets import UserSecretsClient

genai.__version__

'1.7.0'

Retrieve the Gemini API key to make futur requests. Please use the secret function from kaggle.

In [3]:
GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

# User Interaction

You can use a video that you uploaded into kaggle or with a youtube video

In [4]:
def get_file_or_youtube_link():
    """
    Asks the user to enter either a local file path or a YouTube link.
    Checks if the file exists locally if a file path is entered.

    Returns:
        str: The file path or YouTube link entered by the user.  Returns None if the file does not exist.
    """

    while True:
        user_input = input("Please enter a local file path or a YouTube link: ")

        # Check if it looks like a file path
        if os.path.exists(user_input):
            return user_input  # Valid local file
        elif "youtube.com" in user_input or "youtu.be" in user_input:
            return user_input # Assume YouTube link if contains 'youtube.com' or 'youtu.be'
        else:
            print("Invalid input. Please enter a valid file path or a YouTube link.")

In [5]:
file_or_url = get_file_or_youtube_link()
if file_or_url:
    print(f"You entered: {file_or_url}")
    if os.path.exists(file_or_url):
        print("This is a valid local file.")
    else:
        print("This is a YouTube link.")
else:
    print("No valid file or link was provided.")

StdinNotImplementedError: raw_input was called, but this frontend does not support input requests.

# Prompt to  analyse

In [ ]:
#Create a function that analyse the video uploaded by the user. 
#It has a structured output to get some structure to the output.
#The temperature config is low in order to have an output that fit the output.
def videoanalyse():
    client = genai.Client(
        api_key=GOOGLE_API_KEY,
    )

    model = "gemini-2.0-flash"
    contents = [
        types.Content(
            role="user",
            parts=[
                types.Part.from_uri(
                    file_uri=file_or_url,
                    mime_type="video/*",
                ),
            ],
        ),
    ]
    generate_content_config = types.GenerateContentConfig(
        temperature=0.1,
        response_mime_type="application/json",
        response_schema=genai.types.Schema(
                        type = genai.types.Type.OBJECT,
                        properties = {
                            "Body language": genai.types.Schema(
                                type = genai.types.Type.OBJECT,
                                properties = {
                                    "Eyes contact": genai.types.Schema(
                                        type = genai.types.Type.STRING,
                                    ),
                                },
                            ),
                            "Language": genai.types.Schema(
                                type = genai.types.Type.OBJECT,
                                properties = {
                                    "Word Choice (Lexicon)": genai.types.Schema(
                                        type = genai.types.Type.OBJECT,
                                        properties = {
                                            "Formal vs. Informal Language": genai.types.Schema(
                                                type = genai.types.Type.STRING,
                                            ),
                                            "Euphemisms & Jargon": genai.types.Schema(
                                                type = genai.types.Type.STRING,
                                            ),
                                            "Qualifiers & Hedges": genai.types.Schema(
                                                type = genai.types.Type.STRING,
                                            ),
                                            "Intensifiers": genai.types.Schema(
                                                type = genai.types.Type.STRING,
                                            ),
                                            "Positive vs. Negative Framing": genai.types.Schema(
                                                type = genai.types.Type.STRING,
                                            ),
                                            "Pronoun Usage": genai.types.Schema(
                                                type = genai.types.Type.STRING,
                                            ),
                                        },
                                    ),
                                    "Syntax": genai.types.Schema(
                                        type = genai.types.Type.OBJECT,
                                        properties = {
                                            "Complex vs. Simple Sentences": genai.types.Schema(
                                                type = genai.types.Type.STRING,
                                            ),
                                            "Passive vs. Active Voice": genai.types.Schema(
                                                type = genai.types.Type.STRING,
                                            ),
                                            "Sentence Fragments": genai.types.Schema(
                                                type = genai.types.Type.STRING,
                                            ),
                                            "Tangential Speech": genai.types.Schema(
                                                type = genai.types.Type.STRING,
                                            ),
                                        },
                                    ),
                                },
                            ),
                            "Content Analysis": genai.types.Schema(
                                type = genai.types.Type.OBJECT,
                                properties = {
                                    "Omissions": genai.types.Schema(
                                        type = genai.types.Type.STRING,
                                    ),
                                    "Contradictions": genai.types.Schema(
                                        type = genai.types.Type.STRING,
                                    ),
                                    "Rationalizations & Justifications": genai.types.Schema(
                                        type = genai.types.Type.STRING,
                                    ),
                                    "Story Coherence": genai.types.Schema(
                                        type = genai.types.Type.STRING,
                                    ),
                                },
                            ),
                        },
                    ),
                    system_instruction=[
                types.Part.from_text(text="You are a professinnal psychology analyse"),
            ]
    )

    for chunk in client.models.generate_content_stream(
        model=model,
        contents=contents,
        config=generate_content_config,
    ):
        print(chunk.text, end="")

#We save the output as it can be used to get learning on it.
video_analyses = videoanalyse()

In [ ]:
#This function is the same as the previous. However it is interesting to note that the model will now rank each elements from 1 to 5.
#It provides a easier way to compare different presentation and people.
#Not required for next run

def videoanalysenum():
    client = genai.Client(
        api_key=GOOGLE_API_KEY,
    )

    model = "gemini-2.0-flash"
    contents = [
        types.Content(
            role="user",
            parts=[
                types.Part.from_uri(
                    file_uri=file_or_url,
                    mime_type="video/*",
                ),
            ],
        ),
    ]
    generate_content_config = types.GenerateContentConfig(
        temperature=0.1,
        response_mime_type="application/json",
        response_schema=genai.types.Schema(
                        type = genai.types.Type.OBJECT,
                        properties = {
                            "Body language": genai.types.Schema(
                                type = genai.types.Type.OBJECT,
                                properties = {
                                    "Eyes contact": genai.types.Schema(
                                        type = genai.types.Type.STRING,
                                        enum = ["1", "2", "3", "4", "5"],
                                    ),
                                },
                            ),
                            "Language": genai.types.Schema(
                                type = genai.types.Type.OBJECT,
                                properties = {
                                    "Word Choice (Lexicon)": genai.types.Schema(
                                        type = genai.types.Type.OBJECT,
                                        properties = {
                                            "Formal vs. Informal Language": genai.types.Schema(
                                                type = genai.types.Type.STRING,
                                                enum = ["1", "2", "3", "4", "5"],
                                            ),
                                            "Euphemisms & Jargon": genai.types.Schema(
                                                type = genai.types.Type.STRING,
                                                enum = ["1", "2", "3", "4", "5"],
                                            ),
                                            "Qualifiers & Hedges": genai.types.Schema(
                                                type = genai.types.Type.STRING,
                                                enum = ["1", "2", "3", "4", "5"],
                                            ),
                                            "Intensifiers": genai.types.Schema(
                                                type = genai.types.Type.STRING,
                                                enum = ["1", "2", "3", "4", "5"],
                                            ),
                                            "Positive vs. Negative Framing": genai.types.Schema(
                                                type = genai.types.Type.STRING,
                                                enum = ["1", "2", "3", "4", "5"],
                                            ),
                                            "Pronoun Usage": genai.types.Schema(
                                                type = genai.types.Type.STRING,
                                                enum = ["1", "2", "3", "4", "5"],
                                            ),
                                        },
                                    ),
                                    "Syntax": genai.types.Schema(
                                        type = genai.types.Type.OBJECT,
                                        properties = {
                                            "Complex vs. Simple Sentences": genai.types.Schema(
                                                type = genai.types.Type.STRING,
                                                enum = ["1", "2", "3", "4", "5"],
                                            ),
                                            "Passive vs. Active Voice": genai.types.Schema(
                                                type = genai.types.Type.STRING,
                                                enum = ["1", "2", "3", "4", "5"],
                                            ),
                                            "Sentence Fragments": genai.types.Schema(
                                                type = genai.types.Type.STRING,
                                                enum = ["1", "2", "3", "4", "5"],
                                            ),
                                            "Tangential Speech": genai.types.Schema(
                                                type = genai.types.Type.STRING,
                                                enum = ["1", "2", "3", "4", "5"],
                                            ),
                                        },
                                    ),
                                },
                            ),
                            "Content Analysis": genai.types.Schema(
                                type = genai.types.Type.OBJECT,
                                properties = {
                                    "Omissions": genai.types.Schema(
                                        type = genai.types.Type.STRING,
                                        enum = ["1", "2", "3", "4", "5"],
                                    ),
                                    "Contradictions": genai.types.Schema(
                                        type = genai.types.Type.STRING,
                                        enum = ["1", "2", "3", "4", "5"],
                                    ),
                                    "Rationalizations & Justifications": genai.types.Schema(
                                        type = genai.types.Type.STRING,
                                        enum = ["1", "2", "3", "4", "5"],
                                    ),
                                    "Story Coherence": genai.types.Schema(
                                        type = genai.types.Type.STRING,
                                        enum = ["1", "2", "3", "4", "5"],
                                    ),
                                },
                            ),
                        },
                    ),
            system_instruction=[
                types.Part.from_text(text="You are a professinnal psychology analyse"),
            ]
    )

    for chunk in client.models.generate_content_stream(
        model=model,
        contents=contents,
        config=generate_content_config,
    ):
        print(chunk.text, end="")

videoanalysenum()

In [ ]:
#few shot example to show congruence and incongruence

few_shot_examples = """
examples of Congruence (Matching):
A person says "I'm very excited about this project" with genuine enthusiasm in their voice, a wide smile, open posture, and forward leaning.
Someone expresses sadness about a situation while exhibiting a downcast expression, slumped posture, and speaking in a quiet, monotone voice.
Examples of Incongruence (Mismatching - Potential Red Flags):
A person says "I'm not nervous at all" while visibly sweating, fidgeting, and avoiding eye contact. (Possible deception or discomfort).
Someone delivers an apology while smiling slightly and maintaining a closed-off posture. (Potentially insincere apology).
A person describes a positive experience using hesitant language (qualifiers, pauses) while exhibiting microexpressions of discomfort or disgust. (Conflicted feelings or concealed negative experience).
A person states confidently their interest in a conversation but consistently turns their body away.

Analyze the following video for congruence/incongruence basExed on the examples above:
"""

#We bring everything together to have a generic answer
Sysinst = """You are a professional bodylangue analyst. 
Give a global description of the video about his topic and elements

{few_shot_examples} give potential congruence on this video.

For each point on {video_analyses} give idea to improve and work on it

"""



In [ ]:
def General():
    client = genai.Client(
        api_key=GOOGLE_API_KEY,
    )

    model = "gemini-2.0-flash"
    contents = [
        types.Content(
            role="user",
            parts=[
                types.Part.from_uri(
                    file_uri=file_or_url,
                    mime_type="video/*",
                )
            ],
        ),
    ]
    generate_content_config = types.GenerateContentConfig(
        temperature=1.2,
        response_mime_type="text/plain",
        system_instruction=[
            types.Part.from_text(text=Sysinst),
        ],
    )

    for chunk in client.models.generate_content_stream(
        model=model,
        contents=contents,
        config=generate_content_config,
    ):
        print(chunk.text, end="")

if __name__ == "__main__":
    General()
